In [1]:
import pandas as pd
import os

In [2]:
blast_logs = []
megablast_logs = [os.path.join('logs',log_file) for log_file in os.listdir('logs')]
blastn_logs = [os.path.join('blastn_logs',log_file) for log_file in os.listdir('blastn_logs')]
log_files = megablast_logs + blastn_logs
for log_file in log_files:
    if 'loop_' in log_file or 'loopn_' in log_file:
        task_id = log_file.split('.')[0].split('_')[1]
        with open(log_file) as read_log:
            log_entry = {'task_id': task_id}
            for line in read_log:
                if line.startswith('+'):
                    line_split = line.strip().split(' ')
                    log_entry['start_month'] = line_split[2]
                    log_entry['start_day'] = line_split[3]
                    log_entry['start_year'] = line_split[6]
                    log_entry['start_time'] = line_split[4]
                    log_entry['queue'] = line_split[11]
                    log_entry['job_id'] = line_split[13].split('=')[1]
                    log_entry['node'] = line_split[15]
                elif line.startswith('='):
                    line_split = line.strip().split(' ')
                    log_entry['end_month'] = line_split[2]
                    log_entry['end_day'] = line_split[3]
                    log_entry['end_year'] = line_split[6]
                    log_entry['end_time'] = line_split[4]
                    if line_split[8].split('_')[0] == 'blast':
                        log_entry['task'] = 'megablast'
                        log_entry['seq_count'] = 300
                    elif line_split[8].split('_')[0] == 'blastn':
                        log_entry['task'] = 'blastn'
                        log_entry['seq_count'] = 100
            blast_logs.append(log_entry)

In [3]:
log_df = pd.DataFrame(blast_logs)
log_df.head()

,end_day,end_month,end_time,end_year,job_id,node,queue,seq_count,start_day,start_month,start_time,start_year,task,task_id
0,8,Apr,17:32:59,2020,7636651,compute-64-09,sThC.q,300,8,Apr,17:31:30,2020,megablast,899
1,8,Apr,17:33:51,2020,7636651,compute-81-07,sThC.q,300,8,Apr,17:29:29,2020,megablast,801
2,8,Apr,17:21:47,2020,7636651,compute-81-30,sThC.q,300,8,Apr,17:19:17,2020,megablast,372
3,8,Apr,17:15:51,2020,7636651,compute-43-24,sThC.q,300,8,Apr,17:14:29,2020,megablast,141
4,8,Apr,17:26:35,2020,7636651,compute-81-30,sThC.q,300,8,Apr,17:24:16,2020,megablast,602


In [4]:
log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3998 entries, 0 to 3997
Data columns (total 14 columns):
end_day        3998 non-null object
end_month      3998 non-null object
end_time       3998 non-null object
end_year       3998 non-null object
job_id         3998 non-null object
node           3998 non-null object
queue          3998 non-null object
seq_count      3998 non-null int64
start_day      3998 non-null object
start_month    3998 non-null object
start_time     3998 non-null object
start_year     3998 non-null object
task           3998 non-null object
task_id        3998 non-null object
dtypes: int64(1), object(13)
memory usage: 437.4+ KB


In [5]:
log_df['start_dt'] = pd.to_datetime(log_df['start_month'] + ' ' + \
                                    log_df['start_day'] + ' ' + \
                                    log_df['start_year'] + ' ' + \
                                    log_df['start_time'])
log_df['end_dt'] = pd.to_datetime(log_df['end_month'] + ' ' + \
                                    log_df['end_day'] + ' ' + \
                                    log_df['end_year'] + ' ' + \
                                    log_df['end_time'])
log_df.head()

,end_day,end_month,end_time,end_year,job_id,node,queue,seq_count,start_day,start_month,start_time,start_year,task,task_id,start_dt,end_dt
0,8,Apr,17:32:59,2020,7636651,compute-64-09,sThC.q,300,8,Apr,17:31:30,2020,megablast,899,2020-04-08 17:31:30,2020-04-08 17:32:59
1,8,Apr,17:33:51,2020,7636651,compute-81-07,sThC.q,300,8,Apr,17:29:29,2020,megablast,801,2020-04-08 17:29:29,2020-04-08 17:33:51
2,8,Apr,17:21:47,2020,7636651,compute-81-30,sThC.q,300,8,Apr,17:19:17,2020,megablast,372,2020-04-08 17:19:17,2020-04-08 17:21:47
3,8,Apr,17:15:51,2020,7636651,compute-43-24,sThC.q,300,8,Apr,17:14:29,2020,megablast,141,2020-04-08 17:14:29,2020-04-08 17:15:51
4,8,Apr,17:26:35,2020,7636651,compute-81-30,sThC.q,300,8,Apr,17:24:16,2020,megablast,602,2020-04-08 17:24:16,2020-04-08 17:26:35


In [6]:
log_df['elapsed_time'] = log_df['end_dt'] - log_df['start_dt']
log_df.head()

,end_day,end_month,end_time,end_year,job_id,node,queue,seq_count,start_day,start_month,start_time,start_year,task,task_id,start_dt,end_dt,elapsed_time
0,8,Apr,17:32:59,2020,7636651,compute-64-09,sThC.q,300,8,Apr,17:31:30,2020,megablast,899,2020-04-08 17:31:30,2020-04-08 17:32:59,00:01:29
1,8,Apr,17:33:51,2020,7636651,compute-81-07,sThC.q,300,8,Apr,17:29:29,2020,megablast,801,2020-04-08 17:29:29,2020-04-08 17:33:51,00:04:22
2,8,Apr,17:21:47,2020,7636651,compute-81-30,sThC.q,300,8,Apr,17:19:17,2020,megablast,372,2020-04-08 17:19:17,2020-04-08 17:21:47,00:02:30
3,8,Apr,17:15:51,2020,7636651,compute-43-24,sThC.q,300,8,Apr,17:14:29,2020,megablast,141,2020-04-08 17:14:29,2020-04-08 17:15:51,00:01:22
4,8,Apr,17:26:35,2020,7636651,compute-81-30,sThC.q,300,8,Apr,17:24:16,2020,megablast,602,2020-04-08 17:24:16,2020-04-08 17:26:35,00:02:19


In [7]:
log_df['task'].unique()

array(['megablast', 'blastn'], dtype=object)

In [8]:
log_df[log_df['task'] == 'megablast']['elapsed_time'].sum()

Timedelta('1 days 10:28:01')

In [9]:
log_df[log_df['task'] == 'blastn']['elapsed_time'].sum()

Timedelta('358 days 01:55:30')

In [10]:
log_df[log_df['task'] == 'megablast']['elapsed_time'].describe()

count                      1000
mean     0 days 00:02:04.081000
std      0 days 00:00:46.806188
min             0 days 00:00:31
25%             0 days 00:01:24
50%      0 days 00:02:02.500000
75%      0 days 00:02:37.250000
max             0 days 00:06:30
Name: elapsed_time, dtype: object

In [11]:
log_df[log_df['task'] == 'blastn']['elapsed_time'].describe()

count                      2998
mean     0 days 02:51:59.589726
std      0 days 01:09:52.948327
min             0 days 01:01:31
25%      0 days 01:49:03.500000
50%             0 days 02:08:45
75%      0 days 04:05:12.750000
max             0 days 05:14:20
Name: elapsed_time, dtype: object

In [12]:
log_df[log_df['task'] == 'megablast']['end_dt'].max() - \
log_df[log_df['task'] == 'megablast']['start_dt'].min()

Timedelta('0 days 00:27:17')

In [13]:
log_df[log_df['task'] == 'blastn']['end_dt'].max() - \
log_df[log_df['task'] == 'blastn']['start_dt'].min()

Timedelta('0 days 21:38:46')

In [14]:
total_seqs = 278_775

In [15]:
log_df[log_df['task'] == 'blastn']['elapsed_time'].sum() / total_seqs

Timedelta('0 days 00:01:50.978853')

In [16]:
log_df[log_df['task'] == 'megablast']['elapsed_time'].sum() / total_seqs

Timedelta('0 days 00:00:00.445093')

In [17]:
log_df.to_csv('logs_compared.tsv', index=False, sep='\t')